<a href="https://colab.research.google.com/github/gstvmt/lia1-2023-1/blob/main/Entregas%20-%20GustavoMotaBarros/Entrega%2010%20-%20Aula4%20fastai%20-%20Getting_started_with_NPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
! pip install --no-cache-dir transformers sentencepiece

In [3]:
from google.colab import files
files.upload()

{}

In [4]:
#! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory


In [6]:
creds = '{"username":"gustavomotabarros","key":"4b49ccaa7bfe05c082af9b24e443dc74"}'

In [7]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)


In [8]:
path = Path('us-patent-phrase-to-phrase-matching')

In [9]:
if not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [10]:
!ls {path}

sample_submission.csv  test.csv  train.csv


In [11]:
import pandas as pd

In [12]:
df = pd.read_csv(path/'train.csv')

In [13]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [14]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [15]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [16]:
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [17]:
!pip install datasets

In [18]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [19]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [20]:
model_nm = 'microsoft/deberta-v3-small'

In [21]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm, Use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
tokz.tokenize("Ola mundo!")

['▁Ola', '▁mundo', '!']

In [23]:
tokz.tokenize("Um ornitorrinco é um ornithorhynchus anatinus.")

['▁Um',
 '▁or',
 'nit',
 'or',
 'rin',
 'co',
 '▁é',
 '▁um',
 '▁or',
 'ni',
 'tho',
 'rhynch',
 'us',
 '▁an',
 'at',
 'inus',
 '.']

In [24]:
def tok_func(x): return tokz(x["input"])

In [25]:
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [26]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [27]:
tokz.vocab['▁of']

265

In [28]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [29]:
eval_df = pd.read_csv(path/'test.csv')

In [30]:
eval_df.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [31]:
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [32]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [33]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [43]:
import numpy as np

def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [35]:
from transformers import TrainingArguments,Trainer

In [36]:
bs = 128
epochs = 4
lr = 8e-5

In [37]:
!pip install accelerate

In [38]:
from transformers import TrainingArguments,Trainer
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [39]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)

trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [44]:
trainer.train();

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.022515,0.831228
2,No log,0.021836,0.834697
3,0.013800,0.021081,0.833500
4,0.013800,0.023023,0.830240


Epoch,Training Loss,Validation Loss


In [45]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.50390625],
       [ 0.64697266],
       [ 0.53222656],
       [ 0.39916992],
       [-0.05484009],
       [ 0.55566406],
       [ 0.50976562],
       [-0.05679321],
       [ 0.19873047],
       [ 1.07714844],
       [ 0.23999023],
       [ 0.23730469],
       [ 0.71630859],
       [ 0.82666016],
       [ 0.75537109],
       [ 0.45605469],
       [ 0.31005859],
       [-0.03924561],
       [ 0.68457031],
       [ 0.36083984],
       [ 0.54101562],
       [ 0.21264648],
       [ 0.04489136],
       [ 0.23461914],
       [ 0.54150391],
       [-0.00340652],
       [-0.01715088],
       [-0.0322876 ],
       [-0.00365829],
       [ 0.59570312],
       [ 0.32104492],
       [-0.01428986],
       [ 0.75097656],
       [ 0.4675293 ],
       [ 0.40478516],
       [ 0.19250488]])

In [46]:
preds = np.clip(preds, 0, 1)

In [47]:
preds

array([[0.50390625],
       [0.64697266],
       [0.53222656],
       [0.39916992],
       [0.        ],
       [0.55566406],
       [0.50976562],
       [0.        ],
       [0.19873047],
       [1.        ],
       [0.23999023],
       [0.23730469],
       [0.71630859],
       [0.82666016],
       [0.75537109],
       [0.45605469],
       [0.31005859],
       [0.        ],
       [0.68457031],
       [0.36083984],
       [0.54101562],
       [0.21264648],
       [0.04489136],
       [0.23461914],
       [0.54150391],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.59570312],
       [0.32104492],
       [0.        ],
       [0.75097656],
       [0.4675293 ],
       [0.40478516],
       [0.19250488]])

In [48]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1016